# Lab 4: Introduction to Convolutional Layers

The goal of this lab is to implement convolutional layers in pytorch and illustrate how to use GPU for training CNN

In [1]:
#Import common dependencies
import torch
from torch import nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)

## 2D Convolutions

In [3]:
# question: what would be the desired input shape for the 2D Convolution layer?
desired_shape = (3,15,15) # how does this relate to the input channels?

# generate a random tensor with the desired shape
x_2d = torch.rand(desired_shape)

### Knowing the module

In [ ]:
?nn.Conv2d

Before we create a Convolution layer and see how that works, let's understand what parameters does our 2D convolution layer need and what would be the expected output?

### Questions
- Input channels?
    - Number of slices of 2d matrices
- Output channels?
    - Number of filters. Each filter will result in a slice of matrix by being applied to the input. One filter will be operating on all channels of the input, so the filter shape will be (in_channels, kernel_size, kernel_size).


https://towardsdatascience.com/intuitively-understanding-convolutions-for-deep-learning-1f6f42faee1

### Instantiate a 2D Convolution Layer

In [5]:
# TODO: how many in channels do we have? (check our sample tensor)
conv2d_layer = nn.Conv2d(in_channels=,  # input channels
                         out_channels=5, # output channels
                         kernel_size=3, # kernel size
                         stride=1, # stride
                         padding=1, # padding
                         bias=True
                        ) 

### Number of Parameters in the Convolution Layer

In [ ]:
print(f'Number of parameters in our conv2d_layer are {sum(p.numel() for p in conv2d_layer.parameters())}')

### Output

In [ ]:
x_2d.shape

In [ ]:
x_2d.unsqueeze(0).shape

In [9]:
out = conv2d_layer(x_2d.unsqueeze(0))

### Shape of the output??

In [ ]:
out.shape

### Output shape of torch.Size([1, 8, 5, 5])

In [ ]:
# TODO: create a 2D convolution layer that has an output shape of torch.Size([1, 8, 5, 5]) with the same input tensor
conv2d_layer_2 = # add your code here

out = conv2d_layer_2(x_2d.unsqueeze(0))
out.shape

## Take a look of 1d convolution

In [14]:
# question: What would be the desired input shape for the 1D Convolution layer?
desired_shape = (3,10)

# generate a random tensor of the desired shape
x_1d = torch.rand(desired_shape)

### Instantiate a 1D Convolution Layer

In [15]:
conv1d_layer = nn.Conv1d(in_channels=3,  # input channels
                         out_channels=5, # output channels
                         kernel_size=3, # kernel size
                         stride=1, # stride
                         padding=1, # padding
                         bias=True
                        ) 

### Number of Parameters in the Convolution Layer

In [ ]:
print(f'Number of parameters in our conv1d_layer are {sum(p.numel() for p in conv1d_layer.parameters())}')

In [17]:
conv1d_res = conv1d_layer(x_1d.unsqueeze(0))

In [ ]:
conv1d_res.shape

## Implementing LeNet

In [17]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5) # input channels, output channels, kernel size
        self.pool = nn.MaxPool2d(2, 2) # kernel size, stride
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120) # 16*4*4 comes from the dimensionality of the output tensor before the fully connected layers
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10) # 10 output classes for MNIST digits

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4) # flatten the tensor for the fully connected layer
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform = transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)


In [ ]:

print('Using', device)
model = LeNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    start_time = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    print('Training Epoch: {} - Time: {:.2f} seconds'.format(epoch, time.time() - start_time))

for epoch in range(1, 11): # Train for 10 epochs for demonstration
    train(model, device, train_loader, optimizer, epoch)


In [ ]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

test(model, device, test_loader)

## Create Your Own CNN

In [ ]:
# TODO: see if you can achieve a similar accuracy to LeNet with your own CNN
# make sure to have a different number of convolutional layers in your appraoch